In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import utilities
import sys 
import json
import importlib
sys.path.append('../')
reset = importlib.import_module("utilities.reset")
reset_pipeline = reset.reset_pipeline

# load secrets from a .env file using python-dotenv
from dotenv import load_dotenv
import os
load_dotenv("../../.env")
MY_API_KEY = os.getenv('MY_API_KEY')
MY_API_URL = os.getenv('MY_API_URL')

# import krixik and initialize it with your personal secrets
from krixik import krixik
krixik.init(api_key = MY_API_KEY, 
            api_url = MY_API_URL)

SUCCESS: You are now authenticated.


In [9]:
from utilities.converter import convert_all_notebooks
from utilities.converter import collect_mkdocks_toc

In [14]:
toc_files = collect_mkdocks_toc()

In [29]:
check_file_links(toc_files[0], toc_files)

FAILURE: check_file_links failed for file examples/transcribe/overview.md with exception 'modules/transcribe.mda' is not in list
